# 💧 Streeter–Phelps Water Quality Modeling Notebook

## 📘 Overview

This notebook provides an interactive simulation of the classic **Streeter–Phelps oxygen sag model**, a fundamental tool in environmental engineering and water quality analysis. It allows users to visualize and assess the impact of wastewater discharges on the dissolved oxygen (DO) profile of a river system, accounting for **organic loading**, **deoxygenation**, **reaeration**, and **temperature effects**.

---

## 🎯 Capabilities

- ✅ Simulate DO depletion and recovery using Streeter–Phelps equations  
- ✅ Adjust key environmental and wastewater parameters interactively:
  - Initial BOD load (L₀)
  - Deoxygenation rate (Kₙ)
  - Reaeration rate (Kᵣ)
  - River and wastewater temperatures
  - DO saturation level
  - Minimum DO requirement (regulatory threshold)
  - Flow velocity to estimate downstream distance
- ✅ Automatically computes:
  - 🕒 Time to critical DO (DOₘᵢₙ)
  - 📍 Distance to critical point (km)
  - 📉 Minimum DO concentration
  - ⚠️ Warning if the DO falls below the safe threshold
  - 🔧 Treatment recommendation: required BOD removal for compliance

---

## 🔍 Educational Outcomes

By using this tool, students and engineers can:

- Explore how **temperature**, **flow rate**, and **treatment level** affect DO levels in a river
- Understand the interplay between **decay of organic matter** and **oxygen replenishment**
- Identify the **critical location** where aquatic life is most at risk
- Optimize **BOD removal targets** prior to discharge to ensure environmental compliance

---

## 🧭 Suggested Use

This notebook is ideal for:

- Water quality modeling labs and demonstrations  
- River pollution impact assessment  
- Wastewater design and environmental compliance screening  
- Interactive classroom instruction in environmental fluid mechanics or hydrology

---

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from IPython.display import display

def arrhenius_k(k20, T, theta=1.047):
    return k20 * theta ** (T - 20)

def streeter_phelps(t, L0, kd, kr, DO_sat):
    D = (kd * L0 / (kr - kd)) * (np.exp(-kd * t) - np.exp(-kr * t))
    DO = DO_sat - D
    return D, DO

def plot_sag_critical(
    BOD_input, DO_sat, DO_min_allowed,
    T_ww, T_river, v,
    kd20, kr20
):
    # Time domain
    t = np.linspace(0.01, 30, 500)  # Avoid t=0 to prevent division issues

    # Adjust decay rates using Arrhenius equation
    kd = arrhenius_k(kd20, T_ww)
    kr = arrhenius_k(kr20, T_river)

    # Compute DO deficit and concentration
    D, DO = streeter_phelps(t, BOD_input, kd, kr, DO_sat)

    # Find critical time and distance
    t_crit = t[np.argmax(D)]
    D_crit = np.max(D)
    DO_crit = DO_sat - D_crit
    x_crit = v * t_crit  # assuming v is in km/day

    # Determine if DO_crit meets standard
    status = "✅ Satisfactory" if DO_crit >= DO_min_allowed else "❌ Below Standard"
    
    # Treatment suggestion
    if DO_crit < DO_min_allowed:
        L0_required = ((kr - kd) * (DO_sat - DO_min_allowed)) / (kd * (np.exp(-kd * t_crit) - np.exp(-kr * t_crit)))
        BOD_to_remove = BOD_input - L0_required
        treatment_msg = f"🔧 Recommended: Remove at least {BOD_to_remove:.2f} mg/L BOD before discharge."
    else:
        treatment_msg = "✅ No additional BOD removal required to meet DO standard."

    # Plot
    plt.figure(figsize=(10, 5))
    plt.plot(t, DO, label="DO (mg/L)", color='teal')
    plt.axhline(DO_min_allowed, color='orange', linestyle=':', label=f"Minimum DO ({DO_min_allowed} mg/L)")
    plt.scatter(t_crit, DO_crit, color='red', zorder=5, label=f"Critical DO ≈ {DO_crit:.2f} mg/L")
    plt.title("📉 Streeter–Phelps Oxygen Sag Curve with Critical Point")
    plt.xlabel("Time (days)")
    plt.ylabel("Dissolved Oxygen (mg/L)")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

    print(f"🧭 Critical time: {t_crit:.2f} days")
    print(f"📍 Critical distance: {x_crit:.2f} km")
    print(f"💧 Critical DO: {DO_crit:.2f} mg/L → {status}")
    print(treatment_msg)

# 🎛️ Interactive sliders
interact(
    plot_sag_critical,
    BOD_input=FloatSlider(value=30, min=5, max=60, step=1, description="BOD₀ (mg/L)"),
    DO_sat=FloatSlider(value=9.0, min=6.0, max=12.0, step=0.1, description="DO_sat (mg/L)"),
    DO_min_allowed=FloatSlider(value=5.0, min=2.0, max=7.0, step=0.1, description="Minimum DO"),
    T_ww=FloatSlider(value=25.0, min=10.0, max=35.0, step=1, description="WW Temp (°C)"),
    T_river=FloatSlider(value=20.0, min=5.0, max=30.0, step=1, description="River Temp (°C)"),
    v=FloatSlider(value=1.0, min=0.2, max=5.0, step=0.1, description="Flow speed (km/day)"),
    kd20=FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description="Kₙ @20°C"),
    kr20=FloatSlider(value=0.4, min=0.05, max=1.0, step=0.01, description="Kᵣ @20°C")
)

interactive(children=(FloatSlider(value=30.0, description='BOD₀ (mg/L)', max=60.0, min=5.0, step=1.0), FloatSl…

<function __main__.plot_sag_critical(BOD_input, DO_sat, DO_min_allowed, T_ww, T_river, v, kd20, kr20)>

# 🧪 Educational Module – Streeter–Phelps Water Quality Model

This self-assessment module is designed to accompany the Streeter–Phelps oxygen sag curve simulation. Use these prompts to deepen your understanding of water quality dynamics and critically evaluate your model-based insights.
---
## 🧠 Conceptual Questions

1. **What physical processes are represented by the terms “deoxygenation” and “reaeration” in the Streeter–Phelps model?**

2. **Why does the DO curve initially decrease and then recover downstream from a pollution source?**

3. **How does the river’s flow velocity affect the distance to the critical DO point?**

4. **Why are separate water temperatures used for calculating \( K_d \) and \( K_r \) in the model?**

5. **What assumptions does the Streeter–Phelps model make about river geometry, mixing, and pollutant distribution? When might these assumptions fail?**
---
## 🔍 Reflective Prompts

1. **In your simulation, under what conditions did the critical DO point occur furthest downstream? What physical processes contributed to this?**

2. **What could be the ecological or regulatory consequences if the DO falls below the minimum required level?**

3. **How might increasing river temperature simultaneously impact deoxygenation and reaeration? What does this mean for water quality under climate change?**

4. **How can this model guide treatment decisions or regulatory compliance for a municipal discharge?**

5. **If a river had multiple discharge points, how could this model be extended to account for cumulative oxygen impacts?**
---
## ✅ Quiz Questions

**Q1.** What does \( K_d \) represent in the Streeter–Phelps model?  
A. Reaeration coefficient  
B. Deoxygenation rate constant  
C. Velocity of river flow  
D. Saturation DO concentration  
✅ **Correct Answer:** B

---

**Q2.** The critical point in the DO curve occurs when:  
A. DO deficit is zero  
B. Oxygen input equals consumption  
C. Temperature is highest  
D. Distance from the source is minimal  
✅ **Correct Answer:** B

---

**Q3.** Increasing the temperature of the river will generally:  
A. Lower the saturation DO and increase both \( K_d \) and \( K_r \)  
B. Increase the saturation DO  
C. Decrease deoxygenation and reaeration rates  
D. Eliminate the critical point entirely  
✅ **Correct Answer:** A

---

**Q4.** If the DO at the critical point is lower than the regulatory minimum, the model:  
A. Suggests increasing river depth  
B. Identifies required BOD removal at the discharge  
C. Computes the new saturation DO  
D. Eliminates oxygen deficit  
✅ **Correct Answer:** B

---

**Q5.** Which of the following does not affect the location of the critical DO point?  
A. Initial BOD load  
B. River flow speed  
C. Riverbank slope  
D. Reaeration rate  
✅ **Correct Answer:** C

---